In [534]:
import json
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, AzureOpenAI

In [535]:
client = AzureOpenAI(
            azure_endpoint="https://cullmsouthindia.openai.azure.com/",
            api_key="037155e1b16a432fa836637370eca0e3",
            api_version="2024-02-15-preview",
        )

In [536]:

def read_json_files(directory):
    # List to hold data from all JSON files
    all_data = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'r') as file:
                data = json.load(file)
                all_data.append(data)
    
    return all_data

In [537]:
dataset=read_json_files('/Users/ramnarayanchoudhary/Desktop/UGRIP/updated_ugrip24-ling/ugrip24-ling/datasets/PuzzLing/data_public_data_dev')

In [538]:
len(dataset)

10

In [539]:
dataset[0]

{'source_language': 'dyirbal',
 'target_language': 'english',
 'meta': 'ŋ = ng in hang. ɲ ≈ ni in onion; ȷ is a stop (as d) articulated in the same place in the mouth as ɲ.',
 'train': [['bayi yaɽa ŋunȷaymuŋa baŋgu gurugugu biŋgunman.',
   'Booze is making the man that is always being blamed tired.'],
  ['balan yabu bimabanȷalŋaymuŋa baŋgul yaɽaŋgu guliŋgu ŋunȷaɲu.',
   'The strong man is blaming the mother that is always following death adders.'],
  ['balan waymin bambun baŋgu ȷugaŋgu ȷamiman.',
   'Sugar is making the healthy mother-in-law fat.'],
  ['bala yila wura baŋgul bargandu biŋgundu guniɲu.',
   'The tired wallaby is searching for the little feather.'],
  ['balan malayigara baŋgu garandu biŋgunman.',
   'The smoke is making the scorpion tired.'],
  ['bala gurugu baŋgul ŋumaŋgu munduŋgu dimbaɲu.',
   'The offended father is carrying the booze.'],
  ['bayi midin baŋgun bimaŋgu malayigaraguninaymuŋagu banȷan.',
   'The death adder that is always searching for scorpions is follow

In [540]:
# def question_data_to_puzzle_prompt(core_question_data):
#     BASE_PROMPT = """Here are some expressions in <LANG> (a never-seen-before foreign language) and their translations
# in English:

# <DATA>

# Given the above examples, please translate the following statements  and the translated sentence will be in correct .json fomate without any extra words and bracs and the formate of all the response should be same.
# And the each dataset problem should be sperated and should be with the question and answer properly.\n
# Output formate of solution should be this way , use below as reference for the solution.
# [{'answers': [{'Abun': 'ji gwes', 'English': 'my leg'},
#    {'Abun': 'ji bi gan', 'English': 'my offspring'},
#    {'English': 'my big garden', 'Abun': 'ji sye ngwe'},
#    {'English': "dog's offspring", 'Abun': 'ndar bi gan'}]},
# {

# }

# each and every subdataset should follow the same formate.
# I am requeasting here please provide a correct .json formate so that i can use this .json as further analysis 


# <QUESTIONS>

# """
#     source_language = core_question_data["source_language"]
#     train_data = core_question_data["train"]
#     test_data = core_question_data["test"]
    
#     data_pairs = []
#     for pair in train_data:
#         data_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"\n\t\"english\": \"{pair[1]}\"")
    
#     question_pairs = []
#     for pair in test_data:
#         if pair[2] == ">":  # Translation from source_language to English
#             question_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"")
#         elif pair[2] == "<":  # Translation from English to source_language
#             question_pairs.append(f"\t\"english\": \"{pair[1]}\"")

#     return BASE_PROMPT.replace("<LANG>", source_language).replace("<DATA>", ',\n\n'.join(data_pairs)).replace("<QUESTIONS>", ',\n'.join(question_pairs))

# # Generate and print the puzzle prompt for the given part of the dataset
# print(question_data_to_puzzle_prompt(dataset[0]))


In [541]:
# Given these examples, translate the following statements into the specified languages and provide the output in a clear and concise JSON format:

# {
#   "answers": [
#     {"<LANG1>": "<translation1>", "English": "<translation2>"},
#     {"<LANG1>": "<translation3>", "English": "<translation4>"},
#     ...
#   ]
# }

In [795]:
def question_data_to_puzzle_prompt(core_question_data):
    BASE_PROMPT = """Here are some example translations between English and <LANG>:





Each translation pair should be a dictionary with "<LANG1>" and "English" keys. Multiple pairs should be within a list called "answers".
please remove ```json word , ```plaintext and answer from the response due to this there is problem in response.\n
please use only QUESTIONS for response generation dont print Any other quastion answer information other then  QUESTIONS in response.\n 
please also generate equal number of response to the QUESTIONS which are present and the squence of response should follow the input QUESTIONS correctly, means if the source language always first then response language  \n

<QUESTIONS>
"""
    source_language = core_question_data["source_language"]
    train_data = core_question_data["train"]
    test_data = core_question_data["test"]
    
    data_pairs = []
    for pair in train_data:
        data_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"\n\t\"english\": \"{pair[1]}\"")
    
    question_pairs = []
    for pair in test_data:
        if pair[2] == ">":  # Translation from source_language to English
            question_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"")
        elif pair[2] == "<":  # Translation from English to source_language
            question_pairs.append(f"\t\"english\": \"{pair[1]}\"")

    return BASE_PROMPT.replace("<LANG>", source_language).replace("<DATA>", ',\n\n'.join(data_pairs)).replace("<QUESTIONS>", ',\n'.join(question_pairs))


In [796]:
for i, data in enumerate(dataset):

   print(question_data_to_puzzle_prompt(data) )

Here are some example translations between English and dyirbal:





Each translation pair should be a dictionary with "<LANG1>" and "English" keys. Multiple pairs should be within a list called "answers".
please remove ```json word , ```plaintext and answer from the response due to this there is problem in response.

please use only QUESTIONS for response generation dont print Any other quastion answer information other then  QUESTIONS in response.
 
please also generate equal number of response to the QUESTIONS which are present and the squence of response should follow the input QUESTIONS correctly, means if the source language always first then response language  


	"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.",
	"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.",
	"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.",
	"english": "The little wallaby is looking at the dragonfly.",
	"english": "The aunt that is always being followed is

In [548]:
def qna(core_question_data):
    source_language = core_question_data["source_language"]
    train_data = core_question_data["train"]
    test_data = core_question_data["test"]

    question_pairs = []
    for pair in test_data:
        if pair[2] == ">":  # Translation from source_language to English
            question = f"\"{source_language}\": \"{pair[0]}\""
        elif pair[2] == "<":  # Translation from English to source_language
            question = f"\"english\": \"{pair[1]}\""

        # Check if the question contains unwanted characters
        if '[]' not in question and '\t' not in question:
            question_pairs.append(question)

    return question_pairs


In [549]:
qna(dataset[0])

questions_from_dataset=[]
for i, data in enumerate(dataset):
    questions_from_dataset.append(qna(data))



In [550]:
questions_from_dataset

[['"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."',
  '"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu."',
  '"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan."',
  '"english": "The little wallaby is looking at the dragonfly."',
  '"english": "The aunt that is always being followed is bending the feather."',
  '"english": "The sleeping possum is ignoring the loud noise."',
  '"english": "The caterpillar is searching for the man that is always carrying stones."'],
 ['"english": "The man loves the woman."',
  '"english": "The cat stinks."',
  '"english": "I love (him/her)."',
  '"chickasaw": "Ihooat sahollo."',
  '"chickasaw": "Ofi\'at hilha."',
  '"chickasaw": "Kowi\'ã lhiyohlili."'],
 ['"wambaya": "Alayulujba nguyunyu bungmanyani."',
  '"wambaya": "Yagu gininya."',
  '"wambaya": "Janji darrangguwaji."',
  '"wambaya": "Ngirra nya alanga."',
  '"wambaya": "Daguma nyinga."',
  '"wambaya": "Dirragbi ga balamurrungunya."',
  '"english": "You

In [551]:
for i, data in enumerate(dataset): # please be carefull about the use of the for loop 
    print(f"Puzzle prompt for dataset entry {i}:\n")
    print(question_data_to_puzzle_prompt(data))
    print("\n" + "="*80 + "\n")
    


Puzzle prompt for dataset entry 0:

Here are some example translations between English and dyirbal:

	"dyirbal": "bayi yaɽa ŋunȷaymuŋa baŋgu gurugugu biŋgunman."
	"english": "Booze is making the man that is always being blamed tired.",

	"dyirbal": "balan yabu bimabanȷalŋaymuŋa baŋgul yaɽaŋgu guliŋgu ŋunȷaɲu."
	"english": "The strong man is blaming the mother that is always following death adders.",

	"dyirbal": "balan waymin bambun baŋgu ȷugaŋgu ȷamiman."
	"english": "Sugar is making the healthy mother-in-law fat.",

	"dyirbal": "bala yila wura baŋgul bargandu biŋgundu guniɲu."
	"english": "The tired wallaby is searching for the little feather.",

	"dyirbal": "balan malayigara baŋgu garandu biŋgunman."
	"english": "The smoke is making the scorpion tired.",

	"dyirbal": "bala gurugu baŋgul ŋumaŋgu munduŋgu dimbaɲu."
	"english": "The offended father is carrying the booze.",

	"dyirbal": "bayi midin baŋgun bimaŋgu malayigaraguninaymuŋagu banȷan."
	"english": "The death adder that is alwa

In [552]:
def chat(prompt, model):
    # Ensure the API key is retrieved from the environment variable
    # AZURE_OPENAI_API_KEY = "037155e1b16a432fa836637370eca0e3"
    # api_key = "037155e1b16a432fa836637370eca0e3"
    # if api_key is None:
    #     raise ValueError("AZURE_OPENAI_API_KEY environment variable is not set")
    client = AzureOpenAI(
            azure_endpoint="https://cullmsouthindia.openai.azure.com/",
            api_key="037155e1b16a432fa836637370eca0e3",
            api_version="2024-02-15-preview",
        )
    
    message_text = [{"role": "system", "content": "You are a helpful assistant."}, 
                    {"role": "user", "content": prompt}
                    ]
    completion = client.chat.completions.create(
        model=model,  # model = "deployment_name"
        messages=message_text,
        temperature=0,
        max_tokens=4096,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return completion

In [553]:
def fetch_solution_text(core_question_data):
    prompt = question_data_to_puzzle_prompt(core_question_data)
    model = "gpt4"

    try:
        # Correctly access the response
        response = chat(prompt, model)
        result = response.choices[0].message.content.strip()
        return result

    except Exception as e:
        print(f"Error: {e}")
        return ""

In [554]:
print(fetch_solution_text(dataset[0]))

[
    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},
    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},
    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always sleeping."},
    {"dyirbal": "bala biɲȷiriɲȷu baŋgun bargandu guniɲu.", "English": "The little wallaby is looking at the dragonfly."},
    {"dyirbal": "bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.", "English": "The aunt that is always being followed is bending the feather."},
    {"dyirbal": "bayi bambam baŋgun muŋgaŋgu dibanȷan.", "English": "The sleeping possum is ignoring the loud noise."},
    {"dyirbal": "bayimbam baŋgun ŋumaŋgu biɲȷiriɲȷu guniɲu.", "English": "The caterpillar is searching for the man that is always carrying s

In [702]:
for i, data in enumerate(dataset):
    print(fetch_solution_text(data))



[
    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},
    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},
    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always sleeping."},
    {"dyirbal": "bala biɲȷiriɲȷu baŋgun bargandu guniɲu.", "English": "The little wallaby is looking at the dragonfly."},
    {"dyirbal": "bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.", "English": "The aunt that is always being followed is bending the feather."},
    {"dyirbal": "bala bayimbam baŋgun muŋgaŋgu dibanȷan.", "English": "The sleeping possum is ignoring the loud noise."},
    {"dyirbal": "bala ŋuma baŋgun bilmbalmuŋagu bargan guniɲu.", "English": "The caterpillar is searching for the man that is always carry

In [703]:
# solution_texts = []

# for i, data in enumerate(dataset):
#     solution_texts.append(fetch_solution_text(data))
#     print(i)
#     # solution_texts[i]=fetch_solution_text(data)

In [709]:
solution_texts

['[\t    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},\t    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},\t    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always being slept on."},\t    {"dyirbal": "bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.", "English": "The little wallaby is looking at the dragonfly."},\t    {"dyirbal": "bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.", "English": "The aunt that is always being followed is bending the feather."},\t    {"dyirbal": "bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.", "English": "The sleeping possum is ignoring the loud noise."},\t    {"dyirbal": "bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.", "English": "The caterpillar is searchin

In [711]:

# Clean the output by converting it to a properly formatted string

# def clean_text(text):
#     # Remove newline characters and any extra spaces
#     return text.replace('\n', '').strip()



# Store the cleaned solution texts in a list
solution_texts = []

for data in dataset:
    raw_text = fetch_solution_text(data)
    # cleaned_text = clean_text(raw_text)
    solution_texts.append(raw_text)

# Print the cleaned solution texts
for i, text in enumerate(solution_texts):
    print(f"Solution text for dataset {i+1}:")
    print(text)
    print()  # Add a blank line for readability


Solution text for dataset 1:
[
    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},
    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},
    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always being slept on."},
    {"dyirbal": "bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.", "English": "The little wallaby is looking at the dragonfly."},
    {"dyirbal": "bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.", "English": "The aunt that is always being followed is bending the feather."},
    {"dyirbal": "bayi biɲȷiriɲ baŋgun muŋgaŋgu yabuŋgu dimbaɲu.", "English": "The sleeping possum is ignoring the loud noise."},
    {"dyirbal": "bayimbam baŋgun ŋumaŋgu bilmbalmuŋagu guniɲu.", "English": "The caterpillar is 

In [712]:
solution_texts[4]

'[\n    {"basque": "Neskak problema daki.", "English": "The girl knows the problem."},\n    {"basque": "Emakumearen etxea Europan da.", "English": "The woman\'s house is in Europe."},\n    {"basque": "Nire aitaren liburu berria irakurri dut.", "English": "I have read my father\'s new book."},\n    {"basque": "Aitarekin hotelera joango naiz.", "English": "I will go to the hotel with my father."},\n    {"basque": "Ardo zuri nahi dut.", "English": "I want white wine."},\n    {"basque": "Ardo zuria berria da.", "English": "The white wine is new."},\n    {"basque": "Nire aitaren semea Ingalaterran bizi da.", "English": "My father’s son lives in England."},\n    {"basque": "Nire familia etxe berri bat erosi du.", "English": "My family has bought a new house."}\n]'

In [716]:
l_s=json.loads(solution_texts[6])

In [717]:
len(l_s)

8

In [718]:
l_s

[{'madak': 'lavatbungmenemen', 'English': 'the whole world'},
 {'madak': 'laradi', 'English': 'heart'},
 {'madak': 'lavatkonuna', 'English': 'gardens'},
 {'madak': 'laxantoonan', 'English': 'many things'},
 {'madak': 'levengkatli', 'English': 'grandchildren'},
 {'madak': 'loxot', 'English': 'two parts of day'},
 {'madak': 'lubungkavus', 'English': 'branches'},
 {'English': 'luvaroos', 'madak': 'big branches'}]

basically i want a proper .json formate , make a data list which wil contain all the info and then after travesal into the data

Means now my next step is to form of the a table which contains all the necessary information , so i need to extract solution from data_public_ ref and campare it from the generated solution so for this you have to make a table which contains all the information 


In [581]:
# import pandas as pd
# import json

# # Correct JSON data
# # data_json = '''
# # {
# #  "source_language": "dyirbal",
# #  "target_language": "english",
# #  "meta": "ŋ = ng in hang. ɲ ≈ ni in onion; ȷ is a stop (as d) articulated in the same place in the mouth as ɲ.",
# #  "train": [
# #   ["bayi yaɽa ŋunȷaymuŋa baŋgu gurugugu biŋgunman.", "Booze is making the man that is always being blamed tired."],
# #   ["balan yabu bimabanȷalŋaymuŋa baŋgul yaɽaŋgu guliŋgu ŋunȷaɲu.", "The strong man is blaming the mother that is always following death adders."]
# #   # Add other train data here if needed
# #  ],
# #  "test": [
# #   ["balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "The father that is always being searched for is healing the girl.", ">"],
# #   ["bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "The lizard is searching for the stone that is always being pushed.", ">"]
# #   # Add other test data here if needed
# #  ]
# # }
# # '''

# # Load the JSON data
# data = json.loads(dataset[0])

# # Extract relevant information
# source_language = data["source_language"]
# target_language = data["target_language"]

# # Extract test data
# test_data = data["test"]

# # Placeholder for provided and generated solutions
# # Assuming you have lists of these solutions for demonstration purposes
# provided_solutions = ["Placeholder for provided solution" for _ in test_data]
# generated_solutions = ["Placeholder for generated solution" for _ in test_data]

# # Prepare data for DataFrame
# res = []
# for idx, (source, target, indicator) in enumerate(test_data):
#     row = {
#         "QN Number": idx + 1,
#         "Source Language": source_language,
#         "Target Language": target_language,
#         "Provided Solution": provided_solutions[idx],
#         "Generated Solution": generated_solutions[idx],
#         "Question": f"Test Question {idx + 1}",
#         "Source Sentence": source,
#         "Target Sentence": target
#     }
#     res.append(row)

# # Create DataFrame
# df = pd.DataFrame(res)

# # Display the DataFrame
# print(df)

# # Save to CSV if needed
# df.to_csv('questions_solutions.csv', index=False)


In [582]:
dataset_ref=read_json_files('/Users/ramnarayanchoudhary/Desktop/UGRIP/updated_ugrip24-ling/ugrip24-ling/datasets/PuzzLing/data_public_reference_data_dev')


In [583]:
dataset_ref

[{'source_language': 'dyirbal',
  'target_language': 'english',
  'meta': 'ŋ = ng in hang. ɲ ≈ ni in onion; ȷ is a stop (as d) articulated in the same place in the mouth as ɲ.',
  'train': [['bayi yaɽa ŋunȷaymuŋa baŋgu gurugugu biŋgunman.',
    'Booze is making the man that is always being blamed tired.'],
   ['balan yabu bimabanȷalŋaymuŋa baŋgul yaɽaŋgu guliŋgu ŋunȷaɲu.',
    'The strong man is blaming the mother that is always following death adders.'],
   ['balan waymin bambun baŋgu ȷugaŋgu ȷamiman.',
    'Sugar is making the healthy mother-in-law fat.'],
   ['bala yila wura baŋgul bargandu biŋgundu guniɲu.',
    'The tired wallaby is searching for the little feather.'],
   ['balan malayigara baŋgu garandu biŋgunman.',
    'The smoke is making the scorpion tired.'],
   ['bala gurugu baŋgul ŋumaŋgu munduŋgu dimbaɲu.',
    'The offended father is carrying the booze.'],
   ['bayi midin baŋgun bimaŋgu malayigaraguninaymuŋagu banȷan.',
    'The death adder that is always searching for sc

In [696]:
test_data=dataset_ref[6]["test"]


In [697]:
test_data

[['lavatbungmenemen', 'big group of villages', '>'],
 ['laradi', 'man', '>'],
 ['lavatkonuna', 'big branch', '>'],
 ['laxantoonan', 'many spirits', '>'],
 ['levengkatli', 'eyes', '>'],
 ['loxot', 'place', '>'],
 ['lubungkavus', 'group of white men', '>'],
 ['luvaroos', 'two big vines', '>'],
 ['leveve', 'heart', '<'],
 ['leventaamang', 'gardens', '<'],
 ['laxampanga', 'many things', '<'],
 ['luvuttubuno', 'grandchildren', '<'],
 ['luxonaleng', 'two parts of day', '<'],
 ['lengkonuna', 'branches', '<'],
 ['lempatkonuna', 'big branches', '<']]

In [698]:
question_pairs = []
for idx in range(10):
    test_data=dataset_ref[idx]["test"]
for pair in test_data:
    if pair[2] == ">":  # Translation from source_language to English
        question_pairs.append(f"\t\"{pair[0]}\"")
    elif pair[2] == "<":  # Translation from English to source_language
        question_pairs.append(f"\t\"{pair[1]}\"")

In [587]:
source_language=dataset_ref[0]["source_language"]

In [588]:
source_language

'dyirbal'

In [694]:
solution_pairs = []
for pair in test_data:
    if pair[2] == ">":  # Translation from source_language to English
        solution_pairs.append(f"\t\"{pair[1]}\"")
    elif pair[2] == "<":  # Translation from English to source_language
        solution_pairs.append(f"\t\"{pair[0]}\"")

In [592]:
len(solution_pairs)

7

In [593]:
solution_pairs

['\t"The father that is always being searched for is healing the girl."',
 '\t"The lizard is searching for the stone that is always being pushed."',
 '\t"The man that is always blaming doctors is following the wallaby."',
 '\t"bayi yiriɲȷila baŋgul bargandu wuraŋgu buɽan."',
 '\t"bala yila baŋgun mugunanȷagu banȷalmuŋagu waɽuman."',
 '\t"bala muŋga baŋgul midindu ȷagundu ŋaȷin."',
 '\t"bayi yaɽa dibandimbanaymuŋa baŋgul bayimbambu guniɲu."']

In [699]:
question_pairs

['\t"jaʔaʃwukalaqpoki:k"',
 '\t"xetʃiqʂuʂuŋa:linona:ji"',
 '\t"haxʂuqajtʃipomkat"',
 '\t"ʂuŋa:liʂuto:wqhu:ʔunikat"',
 '\t"Is (the/a) teacher (the/a) liar?"',
 '\t"(The/A) teacher sees (the/a) woman."',
 '\t"This girl does not see my father."',
 '\t"Who is good?"']

Now the nest chanllanging thing is that to extraact the generated solution from the solution_teext 

In [693]:
solution_texts[6]

'[\t  {"madak": "lavatbungmenemen", "English": "the whole world"},\t  {"madak": "laradi", "English": "unknown"},\t  {"madak": "lavatkonuna", "English": "unknown"},\t  {"madak": "laxantoonan", "English": "unknown"},\t  {"madak": "levengkatli", "English": "unknown"},\t  {"madak": "loxot", "English": "unknown"},\t  {"madak": "lubungkavus", "English": "unknown"},\t  {"madak": "luvaroos", "English": "unknown"},\t  {"English": "heart", "madak": "levempeve"},\t  {"English": "gardens", "madak": "unknown"},\t  {"English": "many things", "madak": "unknown"},\t  {"English": "grandchildren", "madak": "lurubuno"},\t  {"English": "two parts of day", "madak": "unknown"},\t  {"English": "branches", "madak": "unknown"},\t  {"English": "big branches", "madak": "unknown"}\t]'

In [682]:
solution_texts

['[\t    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},\t    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},\t    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always being slept on."},\t    {"dyirbal": "bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.", "English": "The little wallaby is looking at the dragonfly."},\t    {"dyirbal": "bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.", "English": "The aunt that is always being followed is bending the feather."},\t    {"dyirbal": "bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.", "English": "The sleeping possum is ignoring the loud noise."},\t    {"dyirbal": "bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.", "English": "The caterpillar is searchin

In [600]:
import json

def extract_content_by_language(data_str, target_language):
    # Parse the JSON-like string data into a list of dictionaries
    try:
        data_list = json.loads(data_str)
    except json.JSONDecodeError:
        return "Invalid JSON data"
    
    # Extract the content for the specified target language
    solutions = [entry.get(target_language, "Content not found") for entry in data_list]

    return solutions


target_language = 'English'

# Extract content for the specified target language
solutions = extract_content_by_language(solution_texts[0], target_language)

#Print or use the solutions
for solution in solutions:
    print(solution)


The healthy boy is searching for the man that is always being ignored.
The big stone is looking at the dragonfly that is always being pushed.
The little wallaby is following the doctor that is always being slept on.
The little wallaby is looking at the dragonfly.
The aunt that is always being followed is bending the feather.
The sleeping possum is ignoring the loud noise.
The caterpillar is searching for the man that is always carrying stones.


In [692]:
print(solution_texts[6])


[	  {"madak": "lavatbungmenemen", "English": "the whole world"},	  {"madak": "laradi", "English": "unknown"},	  {"madak": "lavatkonuna", "English": "unknown"},	  {"madak": "laxantoonan", "English": "unknown"},	  {"madak": "levengkatli", "English": "unknown"},	  {"madak": "loxot", "English": "unknown"},	  {"madak": "lubungkavus", "English": "unknown"},	  {"madak": "luvaroos", "English": "unknown"},	  {"English": "heart", "madak": "levempeve"},	  {"English": "gardens", "madak": "unknown"},	  {"English": "many things", "madak": "unknown"},	  {"English": "grandchildren", "madak": "lurubuno"},	  {"English": "two parts of day", "madak": "unknown"},	  {"English": "branches", "madak": "unknown"},	  {"English": "big branches", "madak": "unknown"}	]


In [662]:
target_language = 'English'

def exteract_answer_from_gen(solution_text,target_language):

   data_list=json.loads(solution_texts[0])

# Extract the generated solutions for the target language
   solution = [entry[target_language] for entry in data_list if target_language in entry]
   return solution

# Print or return the solutions
# for solution in solutions:
#     print(solution)

In [663]:
for a in solution_texts:
    print(a)
    print("___________________")
    print(exteract_answer_from_gen(a, target_language))
    print("&&&&&&&&&&")


[	    {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The healthy boy is searching for the man that is always being ignored."},	    {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking at the dragonfly that is always being pushed."},	    {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The little wallaby is following the doctor that is always being slept on."},	    {"dyirbal": "bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.", "English": "The little wallaby is looking at the dragonfly."},	    {"dyirbal": "bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.", "English": "The aunt that is always being followed is bending the feather."},	    {"dyirbal": "bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.", "English": "The sleeping possum is ignoring the loud noise."},	    {"dyirbal": "bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.", "English": "The caterpillar is searching for the

In [613]:
s_l=json.loads(solution_texts[0])

In [624]:
s_l

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
  'English': 'The healthy boy is searching for the man that is always being ignored.'},
 {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.',
  'English': 'The big stone is looking at the dragonfly that is always being pushed.'},
 {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.',
  'English': 'The little wallaby is following the doctor that is always being slept on.'},
 {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.',
  'English': 'The little wallaby is looking at the dragonfly.'},
 {'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.',
  'English': 'The aunt that is always being followed is bending the feather.'},
 {'dyirbal': 'bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.',
  'English': 'The sleeping possum is ignoring the loud noise.'},
 {'dyirbal': 'bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.',
  'English': 'The caterpillar is searching for the man that

In [660]:
target_language = 'English'
for s in s_l:
    print(s)

    print("___________________")
    print(s.get(target_language))



{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}
___________________
The healthy boy is searching for the man that is always being ignored.
{'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'English': 'The big stone is looking at the dragonfly that is always being pushed.'}
___________________
The big stone is looking at the dragonfly that is always being pushed.
{'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'English': 'The little wallaby is following the doctor that is always being slept on.'}
___________________
The little wallaby is following the doctor that is always being slept on.
{'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.', 'English': 'The little wallaby is looking at the dragonfly.'}
___________________
The little wallaby is looking at the dragonfly.
{'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.', 'English': 'Th

this is the nmost important thing that you have to worl on the conversion of .json,loadd(), becuase its convert into normal data 

In [626]:
solution_list=json.loads(solution_texts[0])
solution_list

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
  'English': 'The healthy boy is searching for the man that is always being ignored.'},
 {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.',
  'English': 'The big stone is looking at the dragonfly that is always being pushed.'},
 {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.',
  'English': 'The little wallaby is following the doctor that is always being slept on.'},
 {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.',
  'English': 'The little wallaby is looking at the dragonfly.'},
 {'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.',
  'English': 'The aunt that is always being followed is bending the feather.'},
 {'dyirbal': 'bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.',
  'English': 'The sleeping possum is ignoring the loud noise.'},
 {'dyirbal': 'bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.',
  'English': 'The caterpillar is searching for the man that

In [659]:
target_language = 'dyirbal'
for a in solution_list:
    print(a)

    print("___________________")
    print(a.get(target_language))

    # print(extract_content_by_language(a, target_language))


{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}
___________________
balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.
{'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'English': 'The big stone is looking at the dragonfly that is always being pushed.'}
___________________
bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.
{'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'English': 'The little wallaby is following the doctor that is always being slept on.'}
___________________
bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.
{'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.', 'English': 'The little wallaby is looking at the dragonfly.'}
___________________
bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.
{'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.', 'English': 'The aunt that is always being followed is bending the feather.'}
__

In [661]:
def extract_content_by_language(entry_dict, target_language):
    content = entry_dict.get(target_language, "Content not found")
    
    return content




In [764]:
entry_dict={'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}
target_language='dyirbal'
#print(extract_content_by_language(entry_dict,target_language))
entry_dict.get(target_language)

'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.'

In [766]:
solution_list=json.loads(solution_texts[0])

In [767]:
solution_list

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
  'English': 'The healthy boy is searching for the man that is always being ignored.'},
 {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.',
  'English': 'The big stone is looking at the dragonfly that is always being pushed.'},
 {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.',
  'English': 'The little wallaby is following the doctor that is always being slept on.'},
 {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.',
  'English': 'The little wallaby is looking at the dragonfly.'},
 {'dyirbal': 'bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.',
  'English': 'The aunt that is always being followed is bending the feather.'},
 {'dyirbal': 'bayi biɲȷiriɲ baŋgun muŋgaŋgu yabuŋgu dimbaɲu.',
  'English': 'The sleeping possum is ignoring the loud noise.'},
 {'dyirbal': 'bayimbam baŋgun ŋumaŋgu bilmbalmuŋagu guniɲu.',
  'English': 'The caterpillar is searching for the man that is always c

In [769]:
target_language='dyirbal'
for s in solution_list:
    print(s.get(target_language))

balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.
bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.
bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.
bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.
bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.
bayi biɲȷiriɲ baŋgun muŋgaŋgu yabuŋgu dimbaɲu.
bayimbam baŋgun ŋumaŋgu bilmbalmuŋagu guniɲu.


In [633]:
s_l_1=[]
for s in solution_texts:
    s_l_1.append(json.loads(s))
    print(json.loads(s))


# solution_list

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}, {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'English': 'The big stone is looking at the dragonfly that is always being pushed.'}, {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'English': 'The little wallaby is following the doctor that is always being slept on.'}, {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.', 'English': 'The little wallaby is looking at the dragonfly.'}, {'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.', 'English': 'The aunt that is always being followed is bending the feather.'}, {'dyirbal': 'bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.', 'English': 'The sleeping possum is ignoring the loud noise.'}, {'dyirbal': 'bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.', 'English': 'The caterpillar is searching for the man that is always carrying 

In [641]:
s_l_1[0]

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
  'English': 'The healthy boy is searching for the man that is always being ignored.'},
 {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.',
  'English': 'The big stone is looking at the dragonfly that is always being pushed.'},
 {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.',
  'English': 'The little wallaby is following the doctor that is always being slept on.'},
 {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.',
  'English': 'The little wallaby is looking at the dragonfly.'},
 {'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.',
  'English': 'The aunt that is always being followed is bending the feather.'},
 {'dyirbal': 'bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.',
  'English': 'The sleeping possum is ignoring the loud noise.'},
 {'dyirbal': 'bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.',
  'English': 'The caterpillar is searching for the man that

In [480]:
# import json
# import re

# def extract_content_by_language(data_str, target_language):
#     # Parse the JSON-like string data into a list of dictionaries
#     try:
#         data_list = json.loads(data_str)
#     except json.JSONDecodeError:
#         return "Invalid JSON data"
    
#     # Extract the content for the specified target language
#     solutions = [entry.get(target_language, "Content not found") for entry in data_list]

#     return solutions

# # Example usage
# data_str = '''[
#     {"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.", "English": "The quiet boy is making the healthy father tired."},
#     {"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.", "English": "The big stone is looking for the little lizard."},
#     {"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.", "English": "The possum that is always being followed by the doctor is ignoring the wallaby."},
#     {"dyirbal": "bala biɲȷiriɲȷu baŋgun bargandu guniɲu.", "English": "The little wallaby is looking at the dragonfly."},
#     {"dyirbal": "bala mugunanȷa baŋgul biɲȷiriɲȷu buóan.", "English": "The aunt that is always being followed is bending the feather."},
#     {"dyirbal": "bala waymin baŋgun muŋgaŋgu dimbaɲu.", "English": "The sleeping possum is ignoring the loud noise."},
#     {"dyirbal": "bayimbam baŋgun ŋumaŋgu guniɲu.", "English": "The caterpillar is searching for the man that is always carrying stones."}
# ]'''

# target_language = 'English'

# # Extract content for the specified target language
# solutions = extract_content_by_language(data_str, target_language)

# # Print or use the solutions
# for solution in solutions:
#     print(solution)


The quiet boy is making the healthy father tired.
The big stone is looking for the little lizard.
The possum that is always being followed by the doctor is ignoring the wallaby.
The little wallaby is looking at the dragonfly.
The aunt that is always being followed is bending the feather.
The sleeping possum is ignoring the loud noise.
The caterpillar is searching for the man that is always carrying stones.


In [638]:
solution_list[0]

{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
 'English': 'The healthy boy is searching for the man that is always being ignored.'}

In [642]:
for d in dataset_ref:
    print(d["test"])

[['balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'The father that is always being searched for is healing the girl.', '>'], ['bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'The lizard is searching for the stone that is always being pushed.', '>'], ['bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'The man that is always blaming doctors is following the wallaby.', '>'], ['bayi yiriɲȷila baŋgul bargandu wuraŋgu buɽan.', 'The little wallaby is looking at the dragonfly.', '<'], ['bala yila baŋgun mugunanȷagu banȷalmuŋagu waɽuman.', 'The aunt that is always being followed is bending the feather.', '<'], ['bala muŋga baŋgul midindu ȷagundu ŋaȷin.', 'The sleeping possum is ignoring the loud noise.', '<'], ['bayi yaɽa dibandimbanaymuŋa baŋgul bayimbambu guniɲu.', 'The caterpillar is searching for the man that is always carrying stones.', '<']]
[['Hattakat ihooã hollo.', 'The man loves the woman.', '<'], ["Kowi'at shoha.", 'The cat stinks.', '<'], ['Holloli.', 'I love (

In [643]:
for t in test_data:
    print(t)

['balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'The father that is always being searched for is healing the girl.', '>']
['bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'The lizard is searching for the stone that is always being pushed.', '>']
['bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'The man that is always blaming doctors is following the wallaby.', '>']
['bayi yiriɲȷila baŋgul bargandu wuraŋgu buɽan.', 'The little wallaby is looking at the dragonfly.', '<']
['bala yila baŋgun mugunanȷagu banȷalmuŋagu waɽuman.', 'The aunt that is always being followed is bending the feather.', '<']
['bala muŋga baŋgul midindu ȷagundu ŋaȷin.', 'The sleeping possum is ignoring the loud noise.', '<']
['bayi yaɽa dibandimbanaymuŋa baŋgul bayimbambu guniɲu.', 'The caterpillar is searching for the man that is always carrying stones.', '<']


In [644]:
train_data=dataset_ref[0]["train"]
test_data=dataset_ref[0]["test"]

data_pairs = []
for pair in train_data:
    data_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"\n\t\"english\": \"{pair[1]}\"")
    
question_pairs = []
for pair in test_data:
    if pair[2] == ">":  # Translation from source_language to English
        question_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"")
    elif pair[2] == "<":  # Translation from English to source_language
        question_pairs.append(f"\t\"english\": \"{pair[1]}\"")

In [645]:
question_pairs

['\t"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."',
 '\t"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu."',
 '\t"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan."',
 '\t"english": "The little wallaby is looking at the dragonfly."',
 '\t"english": "The aunt that is always being followed is bending the feather."',
 '\t"english": "The sleeping possum is ignoring the loud noise."',
 '\t"english": "The caterpillar is searching for the man that is always carrying stones."']

In [651]:
for q in question_pairs:
    print("this is the entire input:", q)
    print("@@@@@@@@@@@@@@@@")
    print("this is source_language:",extract_source_language(q))
    print("###############")
    print("this is the main quastion that we are talking:", extract_content(q))
    print("^^^^^^^^^^^^^^")

this is the entire input: 	"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."
@@@@@@@@@@@@@@@@
this is source_language: dyirbal
###############
this is the main quastion that we are talking: balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.
^^^^^^^^^^^^^^
this is the entire input: 	"dyirbal": "bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu."
@@@@@@@@@@@@@@@@
this is source_language: dyirbal
###############
this is the main quastion that we are talking: bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.
^^^^^^^^^^^^^^
this is the entire input: 	"dyirbal": "bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan."
@@@@@@@@@@@@@@@@
this is source_language: dyirbal
###############
this is the main quastion that we are talking: bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.
^^^^^^^^^^^^^^
this is the entire input: 	"english": "The little wallaby is looking at the dragonfly."
@@@@@@@@@@@@@@@@
this is source_language: english
###############
this is the main quastio

In [652]:
import re

def extract_content(entry):
    # Use regex to extract the content part of the string
    match = re.search(r':\s*"([^"]*)"', entry)
    if match:
        # Return the extracted content
        return match.group(1)
    else:
        return "No content found"

# # Example usage:
# entry1 = '\t"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."'
# entry2 = '\t"english": "The little wallaby is looking at the dragonfly."'

# # Extract content from each entry
# content1 = extract_content(entry1)
# content2 = extract_content(entry2)

# print(content1)  # Output: balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.
# print(content2)  # Output: The little wallaby is looking at the dragonfly.


so the extarct_source_language function working well here , so move forward with your approach 

In [753]:
import re

def extract_source_language(entry):
    # Use regex to extract the key from the string
    match = re.match(r'^\s*"(\w+)":', entry)
    if match:
        # Return the source language (convert to lower case for consistency)
        return match.group(1).lower()
    else:
        return "No source language found"

# # Example usage:
# entry1 = '\t"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."'
# entry2 = '\t"english": "The little wallaby is looking at the dragonfly."'

# # Extract source languages from each entry
# source_language1 = extract_source_language(entry1)
# source_language2 = extract_source_language(entry2)

# print(source_language1)  # Output: dyirbal
# print(source_language2)  # Output: english


In [754]:
entry='"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."'
extract_source_language(entry)

'dyirbal'

In [752]:
import re

def extract_source_language(entry):
    # Use regex to extract the key from the string
    match = re.match(r'^\s*"(\w+)":', entry)
    if match:
        # Return the source language (convert to lower case for consistency)
        return match.group(1).lower()
    else:
        return "No source language found"

# Example usage
entry = '"dyirbal": "balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman."'
print(extract_source_language(entry))  # Output should be 'dyirbal'


dyirbal


In [719]:

# all are from dataset_ref ;

for a,d,idx in dataset_ref:
    
    source_language= d["source_language"]
    target_language=d["target_language"]
    test_data=d["test"]
    solution_list=json.loads(solution_texts[idx])

    question_pairs = []
    solution_pairs = []
    for pair in test_data:
        if pair[2] == ">":  # Translation from source_language to English
          question_pairs.append(f"\t\"{source_language}\": \"{pair[0]}\"")
        elif pair[2] == "<":  # Translation from English to source_language
           question_pairs.append(f"\t\"english\": \"{pair[1]}\"")
  
    for pair in test_data:
         if pair[2] == ">":  # Translation from source_language to English
           solution_pairs.append(f"\t\"{pair[1]}\"")
         elif pair[2] == "<":  # Translation from English to source_language
            solution_pairs.append(f"\t\"{pair[0]}\"")
   
   # now i want to use only one of the elements from the solution_list not at once , so that we have to select one at a time such then we need
            # pass once by once parallely in for loop 

    for (i, q), s in zip(question_pairs, solution_list):
        #the below is the only queastion conent 
        q1=extract_content(q)
        s=extract_source_language(q)
        if s==source_language:
           t=target_language
        else:
           t=target_language
         
        A_s=  solution_pairs[i]   # in this i want Actual solution from the test data set
        G_s= s.get(t)



        
        






ValueError: too many values to unpack (expected 3)

In [666]:
solution_list = json.loads(solution_texts[0])

In [667]:
solution_list

[{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.',
  'English': 'The healthy boy is searching for the man that is always being ignored.'},
 {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.',
  'English': 'The big stone is looking at the dragonfly that is always being pushed.'},
 {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.',
  'English': 'The little wallaby is following the doctor that is always being slept on.'},
 {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.',
  'English': 'The little wallaby is looking at the dragonfly.'},
 {'dyirbal': 'bala mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buɽan.',
  'English': 'The aunt that is always being followed is bending the feather.'},
 {'dyirbal': 'bayi waymin baŋgun muŋga yaɽaŋgu dimbaɲu.',
  'English': 'The sleeping possum is ignoring the loud noise.'},
 {'dyirbal': 'bayimbam baŋgun bayi ŋuma baŋgul dibanbilmbalŋaymuŋagu guniɲu.',
  'English': 'The caterpillar is searching for the man that

In [758]:
t_language='dyirbal'
{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}

print( s.get(t_language))
print("done")

None
done


In [741]:
d=dataset_ref[5]

In [790]:

res = []
idx=-1

# Iterate through the dataset_ref
for  d in dataset_ref:
    
    
    source_language = d["source_language"]
    print("i_s",source_language)
    target_language = d["target_language"]
    test_data = d["test"]
    solution_list = json.loads(solution_texts[idx+1])
    idx=idx+1
    print("s_l",solution_list)

    question_pairs = []
    solution_pairs = []
    for pair in test_data:
        if pair[2] == ">":  # Translation from source_language to English
            question_pairs.append(f"\"{source_language}\": \"{pair[0]}\"")
        elif pair[2] == "<":  # Translation from English to source_language
            question_pairs.append(f"\"english\": \"{pair[1]}\"")
    
    for pair in test_data:
        if pair[2] == ">":  # Translation from source_language to English
            solution_pairs.append(pair[1])
        elif pair[2] == "<":  # Translation from English to source_language
            solution_pairs.append(pair[0])
    print("q_p", question_pairs)
    print("s_p",solution_pairs)
    # Iterate through the question_pairs and solution_list
    for q, s in zip(question_pairs, solution_list):
        print("this is q is going insode",q)
        q_content = extract_content(q)
        print("q_ins:",q_content)
        s_language = extract_source_language(q)
        print("s_la_in",s_language)
        if s_language == source_language:
            t_language = target_language
        else:
            t_language = source_language
        print("t_la_in",t_language)
        A_s = solution_pairs[question_pairs.index(q)]  # Actual solution from the test dataset
        print("the Actual solution :",A_s )
        print(" the s inside the block",s )
        # print("this is try",s.get(t_language))
        G_s=extract_content_by_language(s,t_language)
        print("the gpt Generated solution :",G_s )

        # Create a row for the DataFrame
        row = {
            "source_lang": s_language,
            "Question": q_content,
            "target_lang": t_language,
            "Actual_answe": A_s,
            "Generated_answe": G_s
        }
        res.append(row)
# idx=idx+1

# Create a DataFrame from the result list
res_df = pd.DataFrame(res)

# Display the DataFrame
print(res_df)


# there is little problem in madak solution pair 

i_s dyirbal
s_l [{'dyirbal': 'balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambunman.', 'English': 'The healthy boy is searching for the man that is always being ignored.'}, {'dyirbal': 'bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.', 'English': 'The big stone is looking at the dragonfly that is always being pushed.'}, {'dyirbal': 'bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymuŋagu banȷan.', 'English': 'The little wallaby is following the doctor that is always being slept on.'}, {'dyirbal': 'bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.', 'English': 'The little wallaby is looking at the dragonfly.'}, {'dyirbal': 'bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.', 'English': 'The aunt that is always being followed is bending the feather.'}, {'dyirbal': 'bayi biɲȷiriɲ baŋgun muŋgaŋgu yabuŋgu dimbaɲu.', 'English': 'The sleeping possum is ignoring the loud noise.'}, {'dyirbal': 'bayimbam baŋgun ŋumaŋgu bilmbalmuŋagu guniɲu.', 'English': 'The caterpillar is searching for the man that is always carry

In [791]:
res_df.head()

,source_lang,Question,target_lang,Actual_answe,Generated_answe
0,dyirbal,balan ɲalŋga baŋgul ŋumaŋgu guniymuŋagu bambun...,english,The father that is always being searched for i...,The healthy boy is searching for the man that ...
1,dyirbal,bala diban bilmbalmuŋa baŋgun biɲȷiriɲȷu guniɲu.,english,The lizard is searching for the stone that is ...,The big stone is looking at the dragonfly that...
2,dyirbal,bayi bargan baŋgul yaɽaŋgu gubimbuluŋunȷanaymu...,english,The man that is always blaming doctors is foll...,The little wallaby is following the doctor tha...
3,english,The little wallaby is looking at the dragonfly.,dyirbal,bayi yiriɲȷila baŋgul bargandu wuraŋgu buɽan.,bala ɲalŋga baŋgun biɲȷiriɲȷu bambunȷu.
4,english,The aunt that is always being followed is bend...,dyirbal,bala yila baŋgun mugunanȷagu banȷalmuŋagu waɽu...,bayi mugunanȷa baŋgul yaɽaŋgu biɲȷiriɲȷu buóan.


In [792]:
res_df.to_csv("puzzling_baseline_eval_v3_gpt4.csv",index=False)

In [793]:
solution_list=json.loads(solution_texts[0])
len(solution_list)

7

In [794]:
len(res_df)

75

so there is total 75 quation from 10 different languaga 